Loading necessory packages

In [1]:
import pandas as pd
import numpy as np

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import warnings
warnings.filterwarnings("ignore") # Ignore warnings

Using TensorFlow backend.
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/trdapps/linux-x86_64/envs/BKV_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/trdapps/linux-x86_6

Loading Train dataset ...

In [2]:
df = pd.read_csv('Train.csv')
print('Length of dataset before preprocessing: ', len(df))
print('\n We also have {} different labels/class including {} \n'.format(len(df['Label'].unique()), df['Label'].unique()))

Length of dataset before preprocessing:  1436

 We also have 20 different labels/class including ['POLITICS' 'HEALTH' 'LAW/ORDER' 'RELIGION' 'FARMING'
 'WILDLIFE/ENVIRONMENT' 'SOCIAL ISSUES' 'SOCIAL' 'OPINION/ESSAY'
 'LOCALCHIEFS' 'WITCHCRAFT' 'ECONOMY' 'SPORTS' 'RELATIONSHIPS' 'TRANSPORT'
 'CULTURE' 'EDUCATION' 'MUSIC' 'ARTS AND CRAFTS' 'FLOODING'] 



In [3]:
print('Each sample in raw dataset include a col that contains 15-16 different sentences...\n')
df.head(1500)

Each sample in raw dataset include a col that contains 15-16 different sentences...



,ID,Text,Label
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER
...,...,...,...
1431,ID_zmTmmEio,Eni Minibus Ati Ali ndi Ufulu Wokweza Mitengo ...,TRANSPORT
1432,ID_znOlIaGQ,Kachali apepesa: Kulankhula motumbwa kuthe An...,POLITICS
1433,ID_znracTjN,Mawu supports non-fiction writers The Malawi ...,EDUCATION
1434,ID_ztdsmmva,Tame Mwawa: Phwete ndiye kudya kwake Sewero l...,SOCIAL ISSUES


Generating new table including refID of news agency, sentence in the news, Label of each sentence

In [4]:
X_data = pd.DataFrame(columns=['refID', 'Sentence', 'Label'])
for idx in range(len(df)):
    tmp = df.loc[idx]['Text'].split('\n')
    for jdx in range(len(tmp)):
        X_data = X_data.append({'refID': df.loc[idx]['ID'],'Sentence': tmp[jdx], 'Label': df.loc[idx]['Label']}, ignore_index=True)

preprocessing all samples by changing them to lowercase and erasing punctuations

In [5]:
print('Applying preprocessing steps : Changing all characters to lower-case and removing punctuation')
X_data.loc[:,"Sentence"] = X_data.Sentence.apply(lambda x : str.lower(x))
X_data.loc[:,"Label"] = X_data.Label.apply(lambda x : str.lower(x))
X_data.loc[:,"Sentence"] = X_data.Sentence.str.replace('[^\w\s]','')
X_data.loc[:,"Sentence"] = X_data.Sentence.apply(lambda x : x.translate(string.punctuation))

Applying preprocessing steps : Changing all characters to lower-case and removing punctuation


changing Labels to indecies ...

In [6]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
X_data.loc[:,"Label"] = encoder.fit_transform(X_data.Label)

In [7]:
X_data

,refID,Sentence,Label
0,ID_AASHwXxg,mwangonde khansala wachinyamata akamati achin...,11
1,ID_AASHwXxg,mbiri ya maphunziro anu ndi yotani maphunziro...,11
2,ID_AASHwXxg,mudayamba bwanji zandale kuyambira ndili wach...,11
3,ID_AASHwXxg,ntchito mukugwira ndi zomwe munkayembekezera ...,11
4,ID_AASHwXxg,masomphenya anu ndi otani pandale ine ndine m...,11
...,...,...,...
16496,ID_zteydTpN,phiri adati zomwe adakambirana akuluakuluwo a...,11
16497,ID_zteydTpN,mmbuyomu kafukufuku yemwe nyuzipepala ya the ...,11
16498,ID_zteydTpN,mchaka cha 2006 mtsogoleri wa dziko lino pete...,11
16499,ID_zteydTpN,pamsonkhanowo padali a zipani zosiyanasiyana ...,11


split the dataset into training and validation datasets 

In [8]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X_data['Sentence'], X_data['Label'])

approach 1 : encoding words using raw term count features for LDA.

In [9]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data['Sentence'])
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

approach 2 : encoding words using rExtracting tf-idf features for NMF

In [10]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(X_data['Sentence'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

In [11]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    return metrics.accuracy_score(predictions, valid_y)

In [12]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print('NB, Count Vectors: ', accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('NB, N-Gram Vectors: ', accuracy)


NB, Count Vectors:  0.4541929229277751
NB, N-Gram Vectors:  0.3679108095007271


In [ ]:
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print('SVM, Count Vectors: ', accuracy)

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('SVM, N-Gram Vectors: ', accuracy)

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print('RF, Count Vectors: ', accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print('RF, N-Gram Vectors: ', accuracy)

In [ ]:
#